# SW정글 week03 컴퓨팅 사고로의 전환
## <DFS!>
220415

In [139]:
# 11725 트리의 부모 찾기
import sys
sys.setrecursionlimit(1000000)
# input = sys.stdin.readline

N = int(input())
adj_list = [[] for _ in range(N+1)]
for _ in range(N-1):
    a, b = map(int, input().split())
    adj_list[a].append(b)
    adj_list[b].append(a)

root = 1
parents = [0] * (N+1)
parents[root] = root

def dfs(node=root):
    for adj_node in adj_list[node]:
        if not parents[adj_node]:
            parents[adj_node] = node
            dfs(adj_node)
dfs()
for node in range(2, N+1):
    print(parents[node])

In [8]:
# 1707 이분 그래프
import sys
sys.setrecursionlimit(10**7)
# input = sys.stdin.readline

for _ in range(int(input())):
    V, E = map(int, input().split())
    adj_list = [[] for _ in range(V+1)]
    for _ in range(E):
        u, v = map(int, input().split())
        adj_list[u].append(v)
        adj_list[v].append(u)

    how_far = [0]*(V+1)
    is_bipartite = True
    def dfs(node):
        global is_bipartite
        if is_bipartite:
            for adj_node in adj_list[node]:
                if not how_far[adj_node]:
                    how_far[adj_node] = how_far[node] + 1
                    dfs(adj_node)

                elif not (how_far[node] - how_far[adj_node])%2: # 사이클이 홀수이면
                    is_bipartite = False
                    return
                    
    for node in range(1, V+1): # 그래프가 한 덩어리가 아닌 경우
        if not is_bipartite:
            break
        if not how_far[node]:
            root = node
            how_far[root] = 1
            dfs(root)
    print('YES') if is_bipartite else print('NO')

YES
NO


In [16]:
# 21606 아침 산책
from collections import deque
import sys
sys.setrecursionlimit(1000000)
# input = sys.stdin.readline

N = int(input())
inner_outer = input().rstrip()
adj_list = [[] for _ in range(N+1)]
for _ in range(N-1):
    u, v = map(int, input().split())
    adj_list[u].append(v)
    adj_list[v].append(u)

def n_comb_2(n):
    return n*(n-1)//2

total_cnt = [0]
visited = [0]*(N+1)
root_queue = deque()

def how_many_path(root):
    now_cnt = []
    def dfs(node=root, group_num=None):
        for adj_node in adj_list[node]:
            if not visited[adj_node]:
                visited[adj_node] = 1
                if inner_outer[adj_node-1] == '0': # 실외
                    if group_num is None:
                        now_cnt.append(0)
                        dfs(adj_node, len(now_cnt)-1)
                    else:
                        dfs(adj_node, group_num)
                else: # 실내
                    if group_num is not None:
                        now_cnt[group_num] += 1
                    else:
                        total_cnt[0] += 1
                    root_queue.append(adj_node)
    dfs()
    
    for cnt in now_cnt:
        total_cnt[0] += cnt + n_comb_2(cnt)

for node in range(1, N+1):
    if inner_outer[node-1] == '1':
        root_queue.append(node)
        visited[node] = 1
        break

while root_queue: # 메모리 줄이려고..
    root = root_queue.popleft()
    how_many_path(root)

print(total_cnt[0]*2)

8


In [43]:
# 14888 연산자 끼워넣기 풀고나니 그냥 백트래킹...
def plus(a, b): return a+b
def minus(a, b): return a-b
def mult(a, b): return a*b
def div(a, b):
    if a < 0:
        return -(-a//b)
    return a//b

N = int(input())
A = list(map(int, input().split()))
operator_cnt = list(map(int, input().split()))
operator = (plus, minus, mult, div)

maxi = [-float('inf')]
mini = [float('inf')]
def dfs(n=0, res=A[0]):
    if n == N-1:
        if res > maxi[0]:
            maxi[0] = res
        if res < mini[0]:
            mini[0] = res
        return
    for i, oper in enumerate(operator):
        if operator_cnt[i]:
            operator_cnt[i] -= 1
            dfs(n+1, oper(res, A[n+1]))
            operator_cnt[i] += 1
dfs()
print(maxi[0])
print(mini[0])

54
-24


In [45]:
# 2573 빙산
import sys
sys.setrecursionlimit(1000000)
# input = sys.stdin.readline
N, M = map(int, input().split()) # 행, 열
sea = [list(map(int, input().split())) for _ in range(N)]

In [49]:
from copy import deepcopy
# memo_sea = deepcopy(sea) # 다시 치기 귀찮으니까
sea = deepcopy(memo_sea)
N, M, sea

(5,
 7,
 [[0, 0, 0, 0, 0, 0, 0],
  [0, 2, 4, 5, 3, 0, 0],
  [0, 3, 0, 2, 5, 2, 0],
  [0, 7, 6, 2, 4, 0, 0],
  [0, 0, 0, 0, 0, 0, 0]])

In [66]:
year = 0
iceberg_cnt = 0
iceberg_coords = []

def water_cnt(coord):
    """coord 위치 주변에 접한 물 수 리턴"""
    row, col = coord
    new_coords = [(row+1, col), (row-1, col), (row, col+1), (row, col-1)]
    w_cnt = 0
    for new_coord in new_coords:
        if 0<=new_coord[0]<N and 0<=new_coord[1]<M and not sea[new_coord[0]][new_coord[1]]:
            w_cnt += 1
    return w_cnt

water_cnt_map = [[None]*M for _ in range(N)]

for row, line in enumerate(sea):
    for col, val in enumerate(line):
        if val:
            iceberg_cnt += 1
            iceberg_coords.append((row, col))
            water_cnt_map[row][col] = water_cnt((row, col))

In [67]:
water_cnt_map, iceberg_coords

([[None, None, None, None, None, None, None],
  [None, 2, 2, 1, 2, None, None],
  [None, 2, None, 1, 0, 3, None],
  [None, 2, 2, 1, 2, None, None],
  [None, None, None, None, None, None, None]],
 [(1, 1),
  (1, 2),
  (1, 3),
  (1, 4),
  (2, 1),
  (2, 3),
  (2, 4),
  (2, 5),
  (3, 1),
  (3, 2),
  (3, 3),
  (3, 4)])